In [ ]:
import pandas as pd

# Creating, Reading and Writing

In [ ]:
# creating a dataframe
pd.DataFrame({'Raei':['Mess food is bad', 'Let\'s go out'], 'Anmol':['It isn\'t always that bad', 'But yes, let\'s go out!']})

In [ ]:
# adding index in place of default values 0, 1, 2, ...
pd.DataFrame({'Ravi':['Mess food is bad', 'Let\'s go out'], 'Anmol':['It isn\'t always that bad', 'Yes, let\'s go out!']}, index=['No Biryani', 'Biryani'])

In [ ]:
# creating a series with user-defined indexes and name
pd.Series([1,2,3,4,5], index = ['Day 1','Day 2','Day 3','Day 4','Day 5'], name = 'Day Schedule')

In [ ]:
# reading an existing csv file for data
wineReviews = pd.read_csv('/kaggle/input/winemagdata130kv3/winemag-data-130k-v2.csv')


In [ ]:
# checking the number of rows and columns in the df
wineReviews.shape

In [ ]:
# fetching the first five rows of the df
wineReviews.head()

In [ ]:
# using the internal index present in csv in place of the one provided by pandas
wineReviews = pd.read_csv('/kaggle/input/winemagdata130kv3/winemag-data-130k-v2.csv', index_col = 0)
wineReviews.head()

# Indexing, Selecting & Assigning

In [ ]:
# native accessors: using the object:accessor analogy like in book:title
# wineReviews.country 
# or
wineReviews['country']

In [ ]:
# native accessors will not work if the column name contains special characters like a space so using the indexing operator is better
wineReviews['country'][0]

Indexing in Pandas

Pandas has its own accessor operators, loc and iloc, which are both row-first and column second as opposed to what we do in native python.

-Index based selection: selecting data based on its numerical position in data

In [ ]:
# selecting the first three entries
wineReviews.iloc[:3,0]

In [ ]:
# selecting only the second and third entry
wineReviews.iloc[1:3,0]

In [ ]:
# passing a list of rows to retrieve
wineReviews.iloc[[0,1,2],0]

In [ ]:
# fetching the last 5 records
wineReviews.iloc[-5:,0]

-Label based selection: here data index value is important and not the position

In [ ]:
wineReviews.loc[0,'country']

iloc is conceptually simpler than loc because it ignores the dataset's indices. When we use iloc we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. loc, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using loc instead. For example, here's one operation that's much easier using loc:

In [ ]:
wineReviews.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]

Choosing between loc and iloc

When choosing or transitioning between loc and iloc, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes. iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Manipulating the index

Label-based selection derives its power from the labels in the index. Index are mutable and we can change them in any waywe see fitchanging the index of data

In [ ]:
wineReviews.set_index('title')

Conditional selection : Suppose that we're interested specifically in better-than-average wines produced in Italy.

In [ ]:
wineReviews.country == 'Italy'
# wineReviews.loc[:,'country']=='Italy'

In [ ]:
# select data with country = Italy
wineReviews.loc[(wineReviews.country == 'Italy')]

In [ ]:
# using multiple conditions in loc
wineReviews.loc[(wineReviews.country == 'Italy') & (wineReviews.points>=90)]

In [ ]:
# using multiple conditions in loc
wineReviews.loc[(wineReviews.country == 'Italy') | (wineReviews.points>=90)]

Using builtin conditional selectors:
1. isin: selects data whose value "is in" a list of values

In [ ]:
wineReviews.loc[wineReviews.country.isin(['Italy','France'])]

2. isnull (or nornull): selects values which are (or are not) empty (NaN)

In [ ]:
wineReviews.loc[wineReviews.price.notnull()]

Assigning data

In [ ]:
wineReviews['critic'] = 'everyone'
wineReviews['critic']

In [ ]:
# or using an iterable value
wineReviews['index_backwards'] = range(len(wineReviews), 0, -1)
wineReviews['index_backwards']

# Summary Functions and Maps

Summary functions: Pandas provides many simple "summary functions" (not an official name) which restructure the data in some useful way describe() is type-aware and gives relevant output.

In [ ]:
wineReviews.loc[:,'points'].describe()

In [ ]:
wineReviews.taster_name.describe()

In [ ]:
# getting the mean() of points
wineReviews.points.mean()

In [ ]:
# finding unique values
wineReviews.taster_name.unique()

List of unique values and how often they occur in the dataset, we can use the value_counts() method:

In [ ]:
wineReviews.taster_name.value_counts()

Maps: A map is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. In data science we often have a need for creating new representations from existing data, or for transforming data from the format it is in now to the format that we want it to be in later. 

1. Maps: map() is the first, and slightly simpler one. 

In [ ]:
# For example, suppose that we wanted to remean the scores the wines received to 0. We can do this as follows:
mean_points = wineReviews.points.mean()
 
#lambda functions are used to apply simple functions fastly on a series/dataframe
wineReviews.points.map(lambda p:p-mean_points)

2. Apply: apply() is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.

In [ ]:
def apply_mean_points(row):
    row.points = row.points - mean_points
    return row
wineReviews.apply(apply_mean_points, axis = 'columns')

In [ ]:
# Note that map() and apply() return new, transformed Series and DataFrames, respectively. They don't modify the original data they're called on.
wineReviews.head()

Pandas provides many common mapping operations as built-ins. For example, here's a faster way of remeaning our points column:

In [ ]:
wineReviews.points - mean_points
wineReviews.points

The original data is preserved In this code we are performing an operation between a lot of values on the left-hand side (everything in the Series) and a single value on the right-hand side (the mean value). Pandas looks at this expression and figures out that we must mean to subtract that mean value from every value in the dataset.

Pandas will also understand what to do if we perform these operations between Series of equal length. For example, an easy way of combining country and region information in the dataset would be to do the following:

In [ ]:
wineReviews.country + " - " + wineReviews.region_1

Finding the wine with the best point/price ratio

In [ ]:
bargain_idx = (wineReviews.points / wineReviews.price).idxmax()
bargain_wine = wineReviews.loc[bargain_idx, 'title']
bargain_wine

# Grouping and Sorting

Maps allow us to transform data in a DataFrame or Series one value at a time for an entire column. However, often we want to group our data, and then do 
something specific to the group the data is in. As you'll learn, we do this with the groupby() operation.

Groupwise analysis:
One function we've been using heavily thus far is the value_counts() function. We can replicate what value_counts() does by doing the following:

In [ ]:
wineReviews.groupby('points').points.count()

In [ ]:
# to get the cheapest wine in each point category, we can use the following code snippet
wineReviews.groupby('points').price.min()

You can think of each group we generate as being a slice of our DataFrame containing only data with values that match. This DataFrame is accessible to us directly using the apply() method, and we can then manipulate the data in any way we see fit. For example, here's one way of selecting the name of the first wine reviewed from each winery in the dataset:

In [ ]:
wineReviews.groupby('winery').apply(lambda df:df.title.iloc[0])

For even more fine-grained control, you can also group by more than one column. For an example, here's how we would pick out the best wine by country and province:

In [ ]:
wineReviews.groupby(['country', 'province']).apply(lambda df : df.loc[df.points.idxmax()])

Another groupby() method worth mentioning is agg(), which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:

In [ ]:
wineReviews.groupby('country').price.agg([len, min, max])

Sorting: Grouping returns data in index order, not in value order. The order of the rows is dependent on the values in the index, not in the data. To get data in the order want it in we can sort it ourselves. The sort_values() method is handy for this.

In [ ]:
countries_reviewed = wineReviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed
# prints countries based on their country and province, ignoring the order in which they appear in the dataframe

In [ ]:
countries_reviewed = countries_reviewed.reset_index()   #resetting the order back to how it was in the dataframe
countries_reviewed

In [ ]:
# sorting the wines based on how many times a country name appears
countries_reviewed.sort_values(by='len', ascending=False)

To sort by index values, use the companion method sort_index(). This method has the same arguments and default order:

In [ ]:
countries_reviewed.sort_index()

Sort by more than one column at a time:

In [ ]:
countries_reviewed.sort_values(by=['country','len'])

# Data Types and Missing Values

In this tutorial, you'll learn how to investigate data types within a DataFrame or Series. You'll also learn how to find and replace entries.

Dtypes
The data type for a column in a DataFrame or a Series is known as the dtype.
You can use the dtype property to grab the type of a specific column.

In [ ]:
wineReviews.price.dtype  # returns the dtype of price column
wineReviews.dtypes  # returns the dtype of each column in the dataframe

One peculiarity to keep in mind (and on display very clearly here) is that columns consisting entirely of strings do not get their own type; they are instead given the object type.

It's possible to convert a column of one type into another wherever such a conversion makes sense by using the astype() function.

In [ ]:
wineReviews.points.astype('float64')  # converts the dtype of points from int64 to float64

A dataframe or series index has its own dtype

In [ ]:
wineReviews.index.dtype

Missing data: Entries missing values are given the value NaN, short for "Not a Number". For technical reasons these NaN values are always of the float64 dtype.
Pandas provides some methods specific to missing data. To select NaN entries you can use pd.isnull() (or its companion pd.notnull()). 

In [ ]:
wineReviews[pd.isnull(wineReviews.country)]

Replacing missing values is a common operation. Pandas provides a really handy method for this problem: fillna(). fillna() provides a few different strategies for mitigating such data. For example, we can simply replace each NaN with an "Unknown":

In [ ]:
wineReviews.region_2.fillna("Unknown")

Or we could fill each missing value with the first non-null value that appears sometime after the given record in the database. This is known as the backfill strategy.

Alternatively, we may have a non-null value that we would like to replace. For example, suppose that since this dataset was published, reviewer Kerin O'Keefe has changed her Twitter handle from @kerinokeefe to @kerino. One way to reflect this in the dataset is using the replace() method:

In [ ]:
wineReviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

The replace() method is worth mentioning here because it's handy for replacing missing data which is given some kind of sentinel value in the dataset: things like "Unknown", "Undisclosed", "Invalid", and so on.

# Renaming and Combining

Oftentimes data will come to us with column names, index names, or other naming conventions that we are not satisfied with. In that case, you'll learn how to use pandas functions to change the names of the offending entries to something better.
You'll also explore how to combine data from multiple DataFrames and/or Series.

Renaming: The first function we'll introduce here is rename(), which lets you change index names and/or column names. For example, to change the points column in our dataset to score, we would do:

In [ ]:
wineReviews.rename(columns={'points':'score'})

rename() lets you rename index or column values by specifying a index or column keyword parameter, respectively. It supports a variety of input formats, but usually a Python dictionary is the most convenient. Here is an example using it to rename some elements of the index.

In [ ]:
wineReviews.rename(index={0:'first', 1:'second'})

You'll probably rename columns very often, but rename index values very rarely. For that, set_index() is usually more convenient.
Both the row index and the column index can have their own name attribute. The complimentary rename_axis() method may be used to change these names. 

In [ ]:
# we change the name of the axes, from 0 and 1 to "wines" and "fields"
wineReviews.rename_axis("wines", axis='rows').rename_axis("fields", axis='columns')

Combining: When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series in non-trivial ways. Pandas has three core methods for doing this. In order of increasing complexity, these are concat(), join(), and merge(). Most of what merge() can do can also be done more simply with join(), so we will omit it and focus on the first two functions here.

The simplest combining method is concat(). Given a list of elements, this function will smush those elements together along an axis.

In [ ]:
canadian_youtube = pd.read_csv("/kaggle/input/youtube-dataset-of-countries/Youtube_data/Countries_data/CAvideos.csv")
british_youtube = pd.read_csv("/kaggle/input/youtube-dataset-of-countries/Youtube_data/Countries_data/GBvideos.csv")
pd.concat([canadian_youtube, british_youtube])

The middlemost combiner in terms of complexity is join(). join() lets you combine different DataFrame objects which have an index in common. For example, to pull down videos that happened to be trending on the same day in both Canada and the UK, we could do the following:

In [ ]:
left = canadian_youtube.set_index(['title', 'trending_date'])
right = british_youtube.set_index(['title', 'trending_date'])
left.join(right, lsuffix='_CAN', rsuffix='_UK')

The lsuffix and rsuffix parameters are necessary here because the data has the same column names in both British and Canadian datasets. If this wasn't true (because, say, we'd renamed them beforehand) we wouldn't need them.